<a href="https://colab.research.google.com/github/rushikeshkishorkhankar/rushikeshkishorkhankar/blob/main/17th_June_Practice_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification Model or Sentiment analysis model using RNN

LSTM Network or extension of RNN.

LSTM RNN, we dont have nodes or filters.

Rather, we have memory blocks that are connected through layers

A block is smarter than a classical neuron and a memory that can hold information across previous time steps or previous layers

Each memory block is made of gates and these gates manage the block's state and a output

a block operates upon an input sequences and each gate within a block uses sigmoid activation method to control whether the gate should be triggered or not, making the change of state and information flowing through the network

3.kinds of gates:
1.input gate: decided which values from the input to update the memory state
2.Outputgate: decides what to output based on input and the memory of the block
3.forget gate: decided what information to throw away from the block

each unit is like a mini state machine where the gates of the units have weights learned during the training procedure.



In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
#set up the input pipeline 
dataset, info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)

train_dataset, test_dataset = dataset['train'],dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePPU8RY/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePPU8RY/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
for example, label in train_dataset.take(1):
  print('text:',example.numpy())
  print('label:',label.numpy())

text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label: 0


In [5]:
#prefetching, input pipeline to make my input in ready to go state

BUFFER_SIZE = 10000
BATCH_SIZE = 64


Initially, buffer will be empty.

Input will be fetched from the train dataset on random basis.

any 10000 sequences will be ectracted from the input dataset and transferred to buffer.

Once the buffer is filled, an element is choosen from the buffer uniformly at random

the idea of this complete prefetching is to separate or decouple the time when the data is produced from the time when it is comnsumed

how many sequences should be peretched by the algorithm?
 at the point of time

 therefore for this we have is auto tune method

 AUTOTUNE method: dynamically decided the input to be prefetched 


In [6]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
for example, label in train_dataset.take(1):
  print('text:',example.numpy())
  print()
  print('label:',label.numpy())

text: [b'When we started watching this series on cable, I had no idea how addictive it would be. Even when you hate a character, you hold back because they are so beautifully developed, you can almost understand why they react to frustration, fear, greed or temptation the way they do. It\'s almost as if the viewer is experiencing one of Christopher\'s learning curves.<br /><br />I can\'t understand why Adriana would put up with Christopher\'s abuse of her, verbally, physically and emotionally, but I just have to read the newspaper to see how many women can and do tolerate such behavior. Carmella has a dream house, endless supply of expensive things, but I\'m sure she would give it up for a loving and faithful husband - or maybe not. That\'s why I watch.<br /><br />It doesn\'t matter how many times you watch an episode, you can find something you missed the first five times. We even watch episodes out of sequence (watch season 1 on late night with commercials but all the language, A&E w

Create a text encoder : TextVectorization layer

1. standardize each sample (lowecasing and punctuation stripping)

2.split each sample into substrings(words): tokenization 

3.recombine of substrings into tokens(ngrams)

4. associate a unique integer value with each token

5. transform each sample or sequence into a vector of integers


this layer will further analyse the dataset, determine the frequency of individual string values or words and will create a vocabulary out of them.

size of the vocab could be open ended or you can retrict the amount of vocab

vocab of 32000 words

70000 words

vocab to 1000 words


In [8]:
VOCAB_SIZE = 1000

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)

encoder.adapt(train_dataset.map(lambda text, label: text))

In [9]:
vocab = np.array(encoder.get_vocabulary())

In [10]:
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [11]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 51,  72, 628, ...,   0,   0,   0],
       [ 48,   7, 140, ...,   0,   0,   0],
       [ 10, 249,   2, ...,   0,   0,   0]])

In [12]:
for n in range(3):
  print('original:', example[n].numpy)
  print("Reverse: ", " ".join(vocab[encoded_example[n]]))

original: <bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'When we started watching this series on cable, I had no idea how addictive it would be. Even when you hate a character, you hold back because they are so beautifully developed, you can almost understand why they react to frustration, fear, greed or temptation the way they do. It\'s almost as if the viewer is experiencing one of Christopher\'s learning curves.<br /><br />I can\'t understand why Adriana would put up with Christopher\'s abuse of her, verbally, physically and emotionally, but I just have to read the newspaper to see how many women can and do tolerate such behavior. Carmella has a dream house, endless supply of expensive things, but I\'m sure she would give it up for a loving and faithful husband - or maybe not. That\'s why I watch.<br /><br />It doesn\'t matter how many times you watch an episode, you can find something you missed the first five times. We even watch episodes out o

CREATE THE MODEL

input layer, text vectorization layer, embedding layer

In [13]:
model = tf.keras.Sequential([
                             encoder,
                             tf.keras.layers.Embedding(
                                 input_dim = len(encoder.get_vocabulary()),
                                 output_dim = 64,
                                 mask_zero=True),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64, activation = 'relu'),
                             tf.keras.layers.Dense(1)
])

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [15]:
#train the model

model.fit(train_dataset,epochs=10,validation_data= test_dataset,validation_steps=30)

Epoch 1/10
391/391 [==============================] - 48s 92ms/step - loss: 0.6349 - accuracy: 0.5734 - val_loss: 0.4818 - val_accuracy: 0.7615
Epoch 2/10
391/391 [==============================] - 37s 94ms/step - loss: 0.4232 - accuracy: 0.8014 - val_loss: 0.3878 - val_accuracy: 0.8250
Epoch 3/10
391/391 [==============================] - 35s 88ms/step - loss: 0.3596 - accuracy: 0.8390 - val_loss: 0.3499 - val_accuracy: 0.8526
Epoch 4/10
391/391 [==============================] - 35s 88ms/step - loss: 0.3313 - accuracy: 0.8561 - val_loss: 0.3475 - val_accuracy: 0.8453
Epoch 5/10
391/391 [==============================] - 37s 93ms/step - loss: 0.3181 - accuracy: 0.8641 - val_loss: 0.3293 - val_accuracy: 0.8531
Epoch 6/10
391/391 [==============================] - 33s 83ms/step - loss: 0.3113 - accuracy: 0.8664 - val_loss: 0.3356 - val_accuracy: 0.8432
Epoch 7/10
391/391 [==============================] - 34s 85ms/step - loss: 0.3070 - accuracy: 0.8702 - val_loss: 0.3219 - val_accuracy:

In [16]:
test_loss, test_acc = model.evaluate(test_dataset)

print(test_acc)

391/391 [==============================] - 19s 49ms/step - loss: 0.3152 - accuracy: 0.8608
0.86080002784729


In [18]:
sample_text = ('The movie was cool. The animation and the graphics were out of the world. I would recommend the movie')
predictions =model.predict(np.array([sample_text]))

In [19]:
print(predictions)

[[1.1089307]]
